# Funções de Perda

O módulo ```nn``` e suas 1001 utilidades, também fornece as implementações das principais funções de perda. Então vamos primeiro importar o ```torch``` e o módulo ```nn``` <br>

In [1]:
import torch
from torch import nn

Antes de tudo, vamos conferir qual dispositivo de hardware está disponível para uso.

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


Vamos trabalhar com o dataset de classificação de vinhos.

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html


In [4]:
from sklearn import datasets

wine =  datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)


(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']



Vamos instanciar um MLP com uma camada escondida e uma camada de saída. <br>

In [6]:
class WineClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(WineClassifier, self).__init__()

        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.out = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()

    def forward(self, x):
        feature = self.relu(self.hidden(x))
        output = self.softmax(self.out(feature))
        return output

input_size = data.shape[1]
hidden_size = 32
out_size = len(wine.target_names)

net = WineClassifier(input_size, hidden_size, out_size).to(device)

In [7]:
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


## Classificação

O primeiro passo é instanciar a função de perda de sua escolha. Trata-se de um problema de classificação com 3 classes, nesse caso a Cross Entropy é a função recomendada, que no PyTorch recebe o nome de *CrossEntropyLoss*: https://pytorch.org/docs/stable/nn.html#crossentropyloss 

**Assim como a rede, as entradas e os rótulos, a função de perda também deve ser carregada na GPU**


In [8]:
criterion = nn.CrossEntropyLoss().to(device)

Antes de aplicar a função de perda, vamos fazer o cast dos dados para tensores e extrair as predições ```y'``` da rede.

In [17]:
x_tensor = torch.from_numpy(data).float().to(device)
y_tensor = torch.from_numpy(target).long().to(device)

print(x_tensor.dtype, y_tensor.dtype)

torch.float32 torch.int64


In [14]:
predicao = net(x_tensor)

# Para cada item tem-se 3 possiveis probabilidades
print(predicao.shape)

torch.Size([178, 3])


Confira as dimensões de ```y``` e ```y'```. Enquanto as predições estão em termos de probabilidades, os rótulos de classificação devem são valores inteiros referentes aos índices das classes.

In [15]:
print(predicao.shape, y_tensor.shape)

torch.Size([178, 3]) torch.Size([178])


As funções de perda implementadas no PyTorch esperam o seguinte padrão de chamada:

```python
loss = criterion(prediction, target)
```

Vale lembrar que cada função de perda possui especificidades quanto às dimensões dos seus parâmetros. Para a Cross Entropy:
* prediction: ```(N, C)```
* target: ```(N,)```

In [20]:
loss = criterion(predicao, y_tensor)
print(loss)

tensor(1.2818, device='cuda:0', grad_fn=<NllLossBackward>)


## Regressão


Vamos trabalhar com o dataset de Diabetes, cujo objetivo é prever a progressão da diabetes em um paciente.

https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset

In [26]:
diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

print(data.shape, target.shape)
print(data[0])
print(target[0])


(442, 10) (442,)
[ 0.03807591  0.05068012  0.06169621  0.02187235 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990842 -0.01764613]
151.0


Implementando o MLP

In [28]:
class DiabetesRegression(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(DiabetesRegression, self).__init__()

        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.out = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()

    def forward(self, x):
        feature = self.relu(self.hidden(x))
        output = self.softmax(self.out(feature))
        return output

input_size = data.shape[1]
hidden_size = 32
out_size = 1

net = DiabetesRegression(input_size, hidden_size, out_size).to(device)

Para solucionar problemas de regressão, as funções de perda correspondentes esperam que ambos o rótulo e a predição tenham **a mesma dimensionalidade**. Não se trata mais de um problema categórico.

Portanto, vamos simular um problema de regressão e aplicar a *MSELoss*<br>
Documentação: https://pytorch.org/docs/stable/nn.html#mseloss

In [31]:
criterion = nn.MSELoss().to(device)

x_tensor = torch.from_numpy(data).float().to(device)
y_tensor = torch.from_numpy(target).float().to(device)

print(x_tensor.shape, y_tensor.shape)


torch.Size([442, 10]) torch.Size([442])


In [36]:
predicao = net(x_tensor)
print(predicao.shape)

loss = criterion(predicao.squeeze(),y_tensor)
print(loss)

torch.Size([442, 1])
tensor(28771.2168, device='cuda:0', grad_fn=<MseLossBackward>)


tensor(151.1335, device='cuda:0')


## Documentação
Veja a documentação para consultar a lista de todas as funções de perda implementadas no PyTorch: <br>
https://pytorch.org/docs/stable/nn.html#loss-functions